# 範例 : (Kaggle)鐵達尼生存預測

## 載入模組

In [70]:
# 做完特徵工程前的所有準備
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

## 讀取資料

In [71]:
data_path = r'C:\Users\jas72\Python_DS\Dataset\Part02\\'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

## 整理
## 1.分割train內目標值to trainy與特徵¶
## 2.train set(拋棄目標值與id)
## 3.測試資料集(擷取並拋棄id)
## 4.整合訓練與測試資料集(因為在做特徵工程時須統一方法)
## 5.平坦化目標值

In [72]:
train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])

## 只取類別值 (object) 型欄位, 存於 object_features 中


In [73]:
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Object Features : {object_features}\n')



5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



## 只留類別型欄位

In [74]:
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


## 觀察欄位相異值數量

In [75]:
df.select_dtypes(include=["object"]).apply(pd.Series.nunique)

Name        1307
Sex            2
Ticket       929
Cabin        187
Embarked       4
dtype: int64

## 對照組 : 標籤編碼 + 邏輯斯迴歸


In [76]:
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())

0.7788748937419742


C:\Users\jas72\Anaconda3\envs\TensorFlow2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jas72\Anaconda3\envs\TensorFlow2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jas72\Anaconda3\envs\TensorFlow2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jas72\Anaconda3\envs\TensorFlow2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jas72\Anaconda3\envs\TensorFlow2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarn

In [77]:
df

,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S
...,...,...,...,...,...
413,"Spector, Mr. Woolf",male,A.5. 3236,None,S
414,"Oliva y Ocana, Dona. Fermina",female,PC 17758,C105,C
415,"Saether, Mr. Simon Sivertsen",male,SOTON/O.Q. 3101262,None,S
416,"Ware, Mr. Frederick",male,359309,None,S


## 查看ticket資料內容分布
### 加上 'Ticket' 欄位的計數編碼
### 第一行 : df.groupby(['Ticket']) 會輸出 df 以 'Ticket' 群聚後的結果, 但因為群聚一類只會有一個值, 因此必須要定義運算
### 例如 df.groupby(['Ticket']).size(), 但欄位名稱會變成 size, 要取別名就需要用語法 df.groupby(['Ticket']).agg({'Ticket_Count':'size'})
### 這樣出來的計數欄位名稱會叫做 'Ticket_Count', 因為這樣群聚起來的 'Ticket' 是 index, 所以需要 reset_index() 轉成一欄
### 因此第一行的欄位, 在第三行按照 'Ticket_Count' 排序後, 最後的 DataFrame 輸出如 Out[5]

In [78]:
#df.groupby(['Ticket'])['Embarked'].size().reset_index()
#df.Ticket.value_counts()
count_df = df.groupby(['Ticket'])['Cabin'].agg({'Ticket_Count':'size'}).reset_index()
count_df.sort_values(by=['Ticket_Count'], ascending=False).head(10)

C:\Users\jas72\Anaconda3\envs\TensorFlow2\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  This is separate from the ipykernel package so we can avoid doing imports until


,Ticket,Ticket_Count
778,CA. 2343,11
104,1601,8
775,CA 2144,8
335,3101295,7
454,347077,7
459,347082,7
847,S.O.C. 14879,7
824,PC 17608,7
123,19950,6
49,113781,6


## 但是上面資料表結果只是 'Ticket' 名稱對應的次數, 要做計數編碼還需要第二行 : 將上表結果與原表格 merge, 合併於 'Ticket' 欄位
## 使用 how='left' 是完全保留原資料表的所有 index 與順序


In [79]:
df = pd.merge(df, count_df, on=['Ticket'], how='left')
df

,Name,Sex,Ticket,Cabin,Embarked,Ticket_Count
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S,1
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C,2
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S,1
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S,2
4,"Allen, Mr. William Henry",male,373450,None,S,1
...,...,...,...,...,...,...
1304,"Spector, Mr. Woolf",male,A.5. 3236,None,S,1
1305,"Oliva y Ocana, Dona. Fermina",female,PC 17758,C105,C,3
1306,"Saether, Mr. Simon Sivertsen",male,SOTON/O.Q. 3101262,None,S,1
1307,"Ware, Mr. Frederick",male,359309,None,S,1


## 'Ticket'計數編碼 + 邏輯斯迴歸

In [80]:
df_temp = pd.DataFrame()
for c in object_features:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
df_temp['Ticket_Count'] = df['Ticket_Count']
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.7811221556805532


C:\Users\jas72\Anaconda3\envs\TensorFlow2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jas72\Anaconda3\envs\TensorFlow2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jas72\Anaconda3\envs\TensorFlow2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jas72\Anaconda3\envs\TensorFlow2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jas72\Anaconda3\envs\TensorFlow2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarn

,Name,Sex,Ticket,Cabin,Embarked,Ticket_Count
0,155,1,720,185,3,1
1,286,0,816,106,0,2
2,523,0,914,185,3,1
3,422,0,65,70,3,2
4,22,1,649,185,3,1


## 'Ticket'特徵雜湊 + 邏輯斯迴歸

In [84]:
# 
df_temp = pd.DataFrame()
for c in object_features:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
# 這邊的雜湊編碼, 是直接將 'Ticket' 的名稱放入雜湊函數的輸出數值, 為了要確定是緊密(dense)特徵, 因此除以10後看餘數
# 這邊的 10 是隨機選擇, 不一定要用 10, 同學可以自由選擇購小的數字試看看. 基本上效果都不會太好
df_temp['Ticket_Hash'] = df['Ticket'].map(lambda x:hash(x) % 10)
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.7788686166721104


C:\Users\jas72\Anaconda3\envs\TensorFlow2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jas72\Anaconda3\envs\TensorFlow2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jas72\Anaconda3\envs\TensorFlow2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jas72\Anaconda3\envs\TensorFlow2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jas72\Anaconda3\envs\TensorFlow2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarn

,Name,Sex,Ticket,Cabin,Embarked,Ticket_Hash
0,155,1,720,185,3,4
1,286,0,816,106,0,2
2,523,0,914,185,3,2
3,422,0,65,70,3,2
4,22,1,649,185,3,2


## 'Ticket'計數編碼 + 'Ticket'特徵雜湊 + 邏輯斯迴歸

In [85]:
df_temp = pd.DataFrame()
for c in object_features:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
df_temp['Ticket_Hash'] = df['Ticket'].map(lambda x:hash(x) % 10)
df_temp['Ticket_Count'] = df['Ticket_Count']
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.7777450211664924


C:\Users\jas72\Anaconda3\envs\TensorFlow2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jas72\Anaconda3\envs\TensorFlow2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jas72\Anaconda3\envs\TensorFlow2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jas72\Anaconda3\envs\TensorFlow2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jas72\Anaconda3\envs\TensorFlow2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarn

,Name,Sex,Ticket,Cabin,Embarked,Ticket_Hash,Ticket_Count
0,155,1,720,185,3,4,1
1,286,0,816,106,0,2,2
2,523,0,914,185,3,2,1
3,422,0,65,70,3,2,2
4,22,1,649,185,3,2,1
